In [5]:
import numpy as np
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize
np.random.seed(666)

In [4]:
production_cost = 200
selling_price = 480
discounted_price = 80

# Demand probability distribution
demand_prob = [1/3, 2/3]
demand_distributions = [
    {"mean": 100_000, "std_dev": 35_000},
    {"mean": 75_000, "std_dev": 22_500},
]

# Function to calculate profit
def calculate_profit(produced, demand, production_cost, selling_price, discounted_price):
    sold = min(produced, demand)
    unsold = max(0, produced - demand)
    
    revenue = sold * selling_price + unsold * discounted_price
    cost = produced * production_cost
    
    return revenue - cost

# Function to sample demand and weather
def sample_demand_and_weather(demand_prob, demand_distributions):
    weather = np.random.choice(len(demand_prob), p=demand_prob)
    distribution = demand_distributions[weather]
    demand = np.random.normal(distribution["mean"], distribution["std_dev"])
    return round(demand), weather

# Generate demand scenarios and corresponding weather
n_simulations = 10_000
demand_scenarios = []
weather_scenarios = []

for _ in range(n_simulations):
    demand, weather = sample_demand_and_weather(demand_prob, demand_distributions)
    demand_scenarios.append(demand)
    weather_scenarios.append(weather)

# Monte Carlo simulation function
def monte_carlo_expected_profit(produced, demand_scenarios, weather_scenarios, production_cost, selling_price, discounted_price):
    profits = []
    for i, demand in enumerate(demand_scenarios):
        production = produced[weather_scenarios[i]]
        profit = calculate_profit(production, demand, production_cost, selling_price, discounted_price)
        profits.append(profit)
    
    return np.mean(profits)

# Objective function to minimize (negative expected profit) for a single production level
def objective_single(produced):
    produced = [produced, produced]
    return -monte_carlo_expected_profit(produced, demand_scenarios, weather_scenarios, production_cost, selling_price, discounted_price)

# Objective function to minimize (negative expected profit) for two production levels
def objective_two(produced):
    return -monte_carlo_expected_profit(produced, demand_scenarios, weather_scenarios, production_cost, selling_price, discounted_price)

# Find optimal production for a single production level
result_single = minimize_scalar(objective_single, bounds=(0, 200_000), method='bounded')
optimal_production_single = round(result_single.x)
max_profit_single = round(-result_single.fun)

print("Single production level decision problem:")
print("Optimal number of snowboards to produce:", optimal_production_single)
print("Expected profit:", max_profit_single)

# Find optimal production for two production levels based on the weather
initial_guess = [optimal_production_single, optimal_production_single]
bounds = [(0, 200_000), (0, 200_000)]
result_two = minimize(objective_two, initial_guess, bounds=bounds)
optimal_production_two = [round(x) for x in result_two.x]
max_profit_two = round(-result_two.fun)

print("\nTwo production levels decision problem (conditional on weather):")
print("Optimal number of snowboards to produce for weather = 1:", optimal_production_two[0])
print("Optimal number of snowboards to produce for weather = 0:", optimal_production_two[1])
print("Expected profit:", max_profit_two)

# Compute the difference in expected profits between the two cases
profit_difference = max_profit_two - max_profit_single

print("\nDifference in expected profits between the two cases:", profit_difference)

Single production level decision problem:
Optimal number of snowboards to produce: 95882
Expected profit: 19209824

Two production levels decision problem (conditional on weather):
Optimal number of snowboards to produce for weather = 1: 118164
Optimal number of snowboards to produce for weather = 0: 86794
Expected profit: 19731318

Difference in expected profits between the two cases: 521494
